# Chapter 1 The Reinforcement Learning Problem

## 1.1 Reinforcement Learning

**Reinforcement learning**:  
a class of solution
methods that work well on the class of problems, and the field that studies these problems and their solution methods. 

**Three most important distinguishing features:**  
1. being closed-loop in an essential way  
1. not having direct instructions as to what actions to take, and where the consequences of actions, including reward signals play out
1. over extended time periods  

__*supervised learning*__: learning from a training set of labeled examples provided by a knowledgable external supervisor  
__*unsupervised learning*__: finding structure hidden in collections of unlabeled data  

**One challenge**: trade-off between exploration and exploitation  
**Another key feature**: explicitly considers the whole problem of a goal-directed agent interacting with an uncertain environment  

## 1.2 Examples

*pass*

## 1.3 Elements of Reinforcement Learning

__*policy*__: defines the learning agent’s way of behaving at a given time *(primary)*

__*reward signal*__: defines the goal in a reinforcement learning problem *(secondary)*

__*value function*__: specifies what is good in the long run *(secondary)*

__*model*__: inferences to be made about how the environment will
behave

## 1.4 Limitations and Scope

**Compared to evolutionary methods**: 
Reinforcement learning involve learning while interacting with the environment. Evolutionary methods ignore much of the useful structure of the reinforcement learning problem: they do not use the fact that the policy they are searching for is a function from states to actions; they do not notice which states an individual passes through during its lifetime, or which actions it selects. 

__*policy gradient methods*__: estimate the directions the parameters should be adjusted in order to most rapidly improve a policy’s performance

## 1.5 An Extended Example: Tic-Tac-Toe

__*temporal-difference learning*__: update to the estimated value
of $s$, denoted $V(s)$, can be written as  
<center>$V(s)\leftarrow V(s)+\alpha[V(s_0)−V(s)]$</center>  
where $\alpha$ is a small positive fraction called the step-size parameter, which influences
the rate of learning.

### My anwser:
#### Exercise 1.1
It would learn more competitive. But would make wrong decision.

#### Exercise 1.2
The states' number are dramatically decreased, and the learning process is accelerated. Of course we should use symmetriies, and it would be the same if it is fully learning.

#### Exercise 1.3
It would fall into a local optimal.

#### Exercise 1.4
*TODO*

#### Exercise 1.5
Just like AlphaGO

## 1.6 Summary
*pass*

## 1.7 History of Reinforcement Learning
*pass*

## 1.8 Bibliographical Remarks
*pass*

In [9]:
import numpy as np
N_ROW = 4
N_SIZE = N_ROW * N_ROW
KEY_BASE = np.array([3**i for i in range(N_SIZE)], dtype=np.int16)
KEY_SIZE = 3**N_SIZE
# side:
# 1  for first player
# -1 for second player
class State(object):
    def __init__(self, s = []):
        if len(s) == N_SIZE:
            self.s_ = np.array(s, dtype=np.int8)
        else:
            self.s_ = np.zeros(N_SIZE, dtype=np.int8)
        self.key_ = np.dot(self.s_+1, KEY_BASE)
        self.keys_ = [self.key_]
    def __getitem__(self, i):
        return self.s_[i]
    def repr_point(self, p):
        if p==0: return '.'
        elif p==1: return 'X'
        elif p==-1: return 'O'
    def __repr__(self):
        return '\n'+'\n'.join([' '.join([self.repr_point(self.s_[r*N_ROW+c])
                    for c in range(N_ROW)]) for r in range(N_ROW)])+'\n'
    def __call__(self):
        return self.s_
    def get(self):
        return self.s_
    def side(self):
        ret = -self.s_.sum()
        if ret==0: return 1
        else: return ret
    def judge(self):
        row_sum = self.s_.reshape(N_ROW,N_ROW).sum(axis=0)
        col_sum = self.s_.reshape(N_ROW,N_ROW).sum(axis=1)
        sum_diag_right=self.s_[np.arange(0, N_SIZE, N_ROW+1)].sum()
        sum_diag_left =self.s_[np.arange(N_ROW-1, N_SIZE-1, N_ROW-1)].sum()
        if row_sum.max() == N_ROW or col_sum.max() == N_ROW\
            or sum_diag_right == N_ROW or sum_diag_left == N_ROW:
            return 1
        elif row_sum.min() == -N_ROW or col_sum.min() == -N_ROW\
            or sum_diag_right == -N_ROW or sum_diag_left == -N_ROW:
            return -1
        return 0
    def blank(self):
        return np.argwhere(self.s_==0).flatten()
    def act(self, action, side=None):
        if action not in list(range(N_SIZE)) \
            or self.s_[action]!=0:
            print('Error action')
            return
        if not side or side not in {-1, 0, 1}:
            side=self.side()
        self.s_[action]=side
        self.key_+=side*KEY_BASE[action]
        self.keys_.append(self.key_)
        return self.judge()
    def redo(self, action):
        if action not in list(range(N_SIZE)) \
            or self.s_[action]==0:
            print('Error action redo')
            return
        self.key_-=self.s_[action]*KEY_BASE[action]
        self.s_[action]=0
        self.keys_.pop()
    def key(self):
        return self.key_
    def keys(self):
        return self.keys_
class Player(object):
    def __init__(self, name = 'Base Player'):
        self.name_ = name
    def move(self, state):
        return None
    def name(self):
        return self.name_
class RandomPlayer(Player):
    def __init__(self, name = 'Random Player'):
        self.name_ = name
    def move(self, state):
        return np.random.choice(state.blank())
class TrainedPlayer(Player):
    def __init__(self, name = 'Trained Player'):
        self.value_ = np.zeros(KEY_SIZE, dtype=np.float32)
        self.explore_rate_ = 0.2
        self.alpha_ = 0.1
        self.name_ = name
        self.greedy_ = False
    def value(self, state):
        return self.value_[state.key()]
    def move(self, state, side=None):
        if not side: side=state.side()
        actions = state.blank()
        if not self.greedy_ and np.random.rand()<self.explore_rate_:
            return np.random.choice(actions)
        values = []
        for a in actions:
            if state.act(a)==side:
                state.redo(a)
                return a
            values.append(self.value_[state.key()])
            state.redo(a)
        if side == 1:
            return actions[np.argmax(values)]
        elif side == -1:
            return actions[np.argmin(values)]
    def backup(self, state, reward):
        self.value_[state.key()]=reward
        for k in state.keys()[-2::-1]:
            self.value_[k] = self.alpha_ * (reward-self.value_[k])
            reward = self.value_[k]
    def train(self, epoch=100, other=None): 
        other = self
        self.greedy_=False
        for i in range(epoch):
            r, s = play_one_round(self, other)
            if r!=0:
                self.backup(s, r)
        self.greedy_=True
def play_one_round(player1, player2):
    s=State()
    while(1):
        r=s.act(player1.move(s))
        if r!=0:
            return r, s
        elif(s.blank().size==0):
            return 0, s
        r=s.act(player2.move(s))
        if r!=0:
            return r, s
        elif(s.blank().size==0):
            return 0, s
def race(player1, player2, play_round=100):
    scores={player1.name():0, player2.name():0, 'draw':0}
    for i in range(play_round):
        r, _ = play_one_round(player1, player2)
        if r==1:
            scores[player1.name()]+=1
        elif r==-1:
            scores[player2.name()]+=1
        else:
            scores['draw']+=1
        player1, player2 = player2, player1
    print(scores)
    print(player1.name()+' win rate '+str(scores[player1.name()]/(scores[player1.name()]+scores[player2.name()])))
    

def cmd_play(player):
    s=State()
    while(1):
        if(s.blank().size==0):
            return
        print(s)
        print(s.repr_point(s.side())+' \'s turn:')
        cmd = input()
        if cmd=='':
            r=s.act(player.move(s))
            if r!=0:
                print(s)
                print('Computer win!')
                return
        elif cmd=='e':
            return
        elif len(cmd)==1:
            r=s.act(int(cmd))
            if r!=0:
                print('You win!')
        elif len(cmd)==2:
            r=s.act((int(cmd[0])-1)*N_ROW+int(cmd[1])-1)
            if r!=0:
                print('You win!')

p = TrainedPlayer()
rp = RandomPlayer()
# race(RandomPlayer('d'), rp, 500)
for i in range(10):
    print('Step %d:'%(i*1000))
    p.train(1000)
    race(p, rp, 500)

Step 0:
{'Random Player': 99, 'draw': 107, 'Trained Player': 294}
Trained Player win rate 0.7480916030534351
Step 1000:
{'Random Player': 111, 'draw': 128, 'Trained Player': 261}
Trained Player win rate 0.7016129032258065
Step 2000:
{'Random Player': 108, 'draw': 142, 'Trained Player': 250}
Trained Player win rate 0.6983240223463687
Step 3000:
{'Random Player': 113, 'draw': 112, 'Trained Player': 275}
Trained Player win rate 0.7087628865979382
Step 4000:
{'Random Player': 116, 'draw': 143, 'Trained Player': 241}
Trained Player win rate 0.6750700280112045
Step 5000:
{'Random Player': 83, 'draw': 138, 'Trained Player': 279}
Trained Player win rate 0.7707182320441989
Step 6000:
{'Random Player': 114, 'draw': 126, 'Trained Player': 260}
Trained Player win rate 0.6951871657754011
Step 7000:
{'Random Player': 81, 'draw': 163, 'Trained Player': 256}
Trained Player win rate 0.7596439169139466
Step 8000:
{'Random Player': 88, 'draw': 157, 'Trained Player': 255}
Trained Player win rate 0.7434402